In [3]:
import pandas as pd
import numpy as np
from boruta import BorutaPy
from IPython.display import display

### Data Prep

In [2]:
df = pd.read_csv('data/aml_df.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

/Users/david/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (15,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
display(df.info())

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Columns: 21492 entries, TARGET USI to Low Risk
dtypes: float64(21413), int64(7), object(72)
memory usage: 22.5+ MB


None

,TARGET USI,Diagnostic ID,Gender,Race,Ethnicity,Age at Diagnosis in Days,First Event,Event Free Survival Time in Days,Vital Status,Overall Survival Time in Days,...,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1,__no_feature,__ambiguous,__alignment_not_unique,Low Risk
0,TARGET-20-PADZCG,09A,Female,Unknown,Not Hispanic or Latino,5325,Relapse,314,Dead,536,...,-0.330849,-1.843229,-0.801464,-0.364707,2.164604,-0.152689,17.12435,14.60608,17.85268,0
1,TARGET-20-PAEAKL,09A,Female,Black or African American,Not Hispanic or Latino,5218,Relapse,134,Dead,163,...,-2.177556,-0.717812,-0.888170,1.754821,0.456577,0.965496,16.79422,15.36352,19.67021,0
2,TARGET-20-PAEERJ,09A,Female,White,Not Hispanic or Latino,5082,Relapse,259,Dead,373,...,0.768241,-0.600677,-1.563662,-0.506252,2.328197,-2.909309,16.91605,14.56523,17.57099,0
3,TARGET-20-PAEFGR,09A,Female,White,Not Hispanic or Latino,549,Relapse,497,Alive,4022,...,1.647430,-3.847206,1.085338,-2.167177,3.119785,-1.816488,17.66439,14.83790,18.38274,1
4,TARGET-20-PAEFGT,03A,Female,White,Not Hispanic or Latino,5452,Relapse,213,Dead,315,...,0.646308,-0.395081,-1.284291,-0.907716,2.450552,-1.553860,17.27285,14.92322,18.33485,0


In [7]:
#holdout validation set
final_val = df.sample(frac=0.2)

#X and y for holdout
final_X = final_val[model_columns]
final_y = final_val.iloc[:, -1]

# training data
data = df.drop(index= final_val.index)
X = data[model_columns]
y = data.iloc[:, -1]

In [8]:
display(X.info())

X.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 110 entries, 0 to 135
Columns: 21404 entries, ENSG00000000003.13 to ENSG00000281912.1
dtypes: float64(21404)
memory usage: 18.0 MB


None

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSG00000281555.1,ENSG00000281571.1,ENSG00000281628.1,ENSG00000281649.1,ENSG00000281691.1,ENSG00000281706.1,ENSG00000281741.1,ENSG00000281789.1,ENSG00000281896.1,ENSG00000281912.1
0,-1.780950,-6.121355,3.640955,3.708995,2.382656,6.391657,2.397881,4.656393,3.848511,4.546833,...,-1.780950,0.132623,-0.682824,5.465605,-0.330849,-1.843229,-0.801464,-0.364707,2.164604,-0.152689
2,-0.600677,-6.927145,4.337290,3.704071,1.457825,5.805410,3.083072,4.702628,5.932859,3.684184,...,-1.858071,-0.470147,1.273137,6.077090,0.768241,-0.600677,-1.563662,-0.506252,2.328197,-2.909309
3,-3.320267,-6.927145,4.883009,2.543106,0.248019,5.705305,-6.927145,3.526833,3.006115,3.489940,...,-1.896496,0.471643,0.210317,6.039166,1.647430,-3.847206,1.085338,-2.167177,3.119785,-1.816488
4,0.104784,-6.927145,4.812658,2.553019,0.438541,3.836235,1.244806,3.590229,4.079561,3.158661,...,-1.795276,1.223349,0.362058,6.544109,0.646308,-0.395081,-1.284291,-0.907716,2.450552,-1.553860
8,-2.671721,-6.927145,5.047465,1.309930,-0.132909,7.996995,-2.008910,5.627308,3.806129,3.344211,...,-4.555115,0.676629,0.392204,4.753428,0.005058,-3.934168,-0.347126,-1.822241,1.796843,-0.999583


# Feature Reduction

### Boruta

In [9]:
from sklearn.ensemble import  RandomForestClassifier

In [11]:
rf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=8, n_jobs=-1)
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, max_iter = 200, random_state=8)
feat_selector.fit(X.values, y.values)

Iteration: 	1 / 200
Confirmed: 	0
Tentative: 	21404
Rejected: 	0
Iteration: 	2 / 200
Confirmed: 	0
Tentative: 	21404
Rejected: 	0
Iteration: 	3 / 200
Confirmed: 	0
Tentative: 	21404
Rejected: 	0
Iteration: 	4 / 200
Confirmed: 	0
Tentative: 	21404
Rejected: 	0
Iteration: 	5 / 200
Confirmed: 	0
Tentative: 	21404
Rejected: 	0
Iteration: 	6 / 200
Confirmed: 	0
Tentative: 	21404
Rejected: 	0
Iteration: 	7 / 200
Confirmed: 	0
Tentative: 	21404
Rejected: 	0
Iteration: 	8 / 200
Confirmed: 	0
Tentative: 	1037
Rejected: 	20367


/Users/david/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	9 / 200
Confirmed: 	63
Tentative: 	974
Rejected: 	20367


/Users/david/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	10 / 200
Confirmed: 	63
Tentative: 	974
Rejected: 	20367


...

/Users/david/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	198 / 200
Confirmed: 	72
Tentative: 	15
Rejected: 	21317


/Users/david/anaconda3/lib/python3.6/site-packages/boruta/boruta_py.py:418: RuntimeWarning: invalid value encountered in greater
  hits = np.where(cur_imp[0] > imp_sha_max)[0]


Iteration: 	199 / 200
Confirmed: 	72
Tentative: 	15
Rejected: 	21317


BorutaPy finished running.

Iteration: 	200 / 200
Confirmed: 	72
Tentative: 	3
Rejected: 	21317


BorutaPy(alpha=0.05,
     estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=65, n_jobs=-1,
            oob_score=False,
            random_state=<mtrand.RandomState object at 0xa20843318>,
            verbose=0, warm_start=False),
     max_iter=200, n_estimators='auto', perc=100,
     random_state=<mtrand.RandomState object at 0xa20843318>,
     two_step=True, verbose=2)

In [15]:
selected = X.values[:, feat_selector.support_]
print(selected.shape)

(110, 72)


In [18]:
boruta_mask = feat_selector.support_
boruta_features = model_columns[boruta_mask]
boruta_df = df[model_columns[boruta_mask]]

### Lasso

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier
from xgboost import XGBClassifier

In [33]:
log_model = LogisticRegression(penalty='l1', solver='saga', max_iter=10000)
kf = KFold(n_splits=5, shuffle=True)

ll_performance = []
model_weights = []
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    log_model.fit(X_train, y_train)
    y_pred = log_model.predict_proba(X_test)
    log_ll = log_loss(y_test, y_pred)
    
    ll_performance.append(log_ll)
    model_weights.append(log_model.coef_)    

In [34]:
print(ll_performance)

average_weight = np.mean(model_weights, axis=0)[0]

[0.4986324708267627, 0.2710345057019458, 0.21950575178068352, 0.20300319506063766, 0.06681012437349124]


In [35]:
def important_gene_mask(columns, coefs):
    mask = coefs != 0
    important_genes = columns[mask[0]]
    print(len(important_genes))
    return important_genes

In [36]:
lasso_k1 = set(important_gene_mask(model_columns, model_weights[0]))
lasso_k2 = set(important_gene_mask(model_columns, model_weights[1]))
lasso_k3 = set(important_gene_mask(model_columns, model_weights[2]))
lasso_k4 = set(important_gene_mask(model_columns, model_weights[3]))
lasso_k5 = set(important_gene_mask(model_columns, model_weights[4]))

448
422
482
550
474


In [38]:
lasso_gene_union = set.union(lasso_k1, lasso_k2, lasso_k3, lasso_k4, lasso_k5)
len(lasso_gene_union)

1091

In [39]:
lasso_gene_intersection = set.intersection(lasso_k1, lasso_k2, lasso_k3, lasso_k4, lasso_k5)
len(lasso_gene_intersection)

109

In [41]:
lasso_columns = list(lasso_gene_union)

In [43]:
lasso_boruta_intersection = set.intersection(set(boruta_features), lasso_gene_intersection)
len(lasso_boruta_intersection)

23

In [44]:
lasso_boruta_intersection

{'ENSG00000078399.14',
 'ENSG00000105997.21',
 'ENSG00000106006.6',
 'ENSG00000116176.6',
 'ENSG00000122592.7',
 'ENSG00000132122.10',
 'ENSG00000132975.7',
 'ENSG00000134569.8',
 'ENSG00000139117.12',
 'ENSG00000147488.10',
 'ENSG00000156968.8',
 'ENSG00000170893.3',
 'ENSG00000172236.15',
 'ENSG00000188626.6',
 'ENSG00000213759.7',
 'ENSG00000230453.8',
 'ENSG00000232431.3',
 'ENSG00000248334.5',
 'ENSG00000253187.2',
 'ENSG00000253293.4',
 'ENSG00000254369.5',
 'ENSG00000275325.3',
 'ENSG00000276141.3'}

In [55]:
gene_name = ['HOXA9', 'HOXA3', 'HOXA6', 'TPSG1', 'HOXA7', 'SPATA6', 'GPR12', 'LRP4',
            'CPNE8', 'ST18', 'MPV17L', 'TRH', 'TPSAB1', 'GOLGA8M', 'GT2B11',
            'ANKRD18B', 'AC055876.1', 'WHAMMP2', 'HOXA10-AS', 'HOXA10', 
            'HOXA-AS3', 'PDCD6IPP1', 'WHAMMP3']

gene_zip = list(zip(lasso_boruta_intersection, gene_name))
gene_zip

[('ENSG00000105997.21', 'HOXA9'),
 ('ENSG00000232431.3', 'HOXA3'),
 ('ENSG00000170893.3', 'HOXA6'),
 ('ENSG00000275325.3', 'TPSG1'),
 ('ENSG00000116176.6', 'HOXA7'),
 ('ENSG00000248334.5', 'SPATA6'),
 ('ENSG00000147488.10', 'GPR12'),
 ('ENSG00000078399.14', 'LRP4'),
 ('ENSG00000254369.5', 'CPNE8'),
 ('ENSG00000134569.8', 'ST18'),
 ('ENSG00000139117.12', 'MPV17L'),
 ('ENSG00000106006.6', 'TRH'),
 ('ENSG00000172236.15', 'TPSAB1'),
 ('ENSG00000132975.7', 'GOLGA8M'),
 ('ENSG00000122592.7', 'GT2B11'),
 ('ENSG00000230453.8', 'ANKRD18B'),
 ('ENSG00000253187.2', 'AC055876.1'),
 ('ENSG00000188626.6', 'WHAMMP2'),
 ('ENSG00000132122.10', 'HOXA10-AS'),
 ('ENSG00000276141.3', 'HOXA10'),
 ('ENSG00000156968.8', 'HOXA-AS3'),
 ('ENSG00000213759.7', 'PDCD6IPP1'),
 ('ENSG00000253293.4', 'WHAMMP3')]

In [57]:
pd.DataFrame(gene_zip)

,0,1
0,ENSG00000105997.21,HOXA9
1,ENSG00000232431.3,HOXA3
2,ENSG00000170893.3,HOXA6
3,ENSG00000275325.3,TPSG1
4,ENSG00000116176.6,HOXA7
5,ENSG00000248334.5,SPATA6
6,ENSG00000147488.10,GPR12
7,ENSG00000078399.14,LRP4
8,ENSG00000254369.5,CPNE8
9,ENSG00000134569.8,ST18


those are the feature deemed most important by both lasso rounds + boruta

In [53]:
boruta_not_lasso = set.difference(set(boruta_features), lasso_gene_union)
len(boruta_not_lasso)

25

25 features were considered important by boruta but not picked up by any of the lasso rounds...why?